(base) root@hudsonPV:/home/tryit/public_html/abv/lib/abv# pip install -e .

how would download_abv() and upload_abv() if they were downloading and uploading from another server that does'nt have ftp but does have ssh and both machines share keys so no password is needed

## SSH-Based File Transfer Functions

For servers with SSH access and shared keys (no password needed), you have several options:

1. **Using `paramiko` library** (Python SSH client)
2. **Using `subprocess` with `scp` commands**
3. **Using `subprocess` with `rsync` over SSH**

Let's implement all three approaches:

In [1]:
# Option 1: Using subprocess with scp (Simple and reliable)

import subprocess
import os

def download_abv_ssh(file_array, host, username, remote_dir="public_html/", 
                     local_dir="./", ssh_key_path=None):
    """Download files from SSH server using scp"""
    
    try:
        for filename in file_array:
            # Build remote path
            remote_path = f"{username}@{host}:{remote_dir}{filename}"
            local_path = os.path.join(local_dir, filename)
            
            # Build scp command
            cmd = ["scp"]
            if ssh_key_path:
                cmd.extend(["-i", ssh_key_path])
            cmd.extend([remote_path, local_path])
            
            print(f"Downloading {filename}...")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✓ {filename} downloaded")
            else:
                print(f"✗ Error downloading {filename}: {result.stderr}")
        
        print("Download process completed!")
        
    except Exception as e:
        print(f"Error: {e}")

def upload_abv_ssh(file_array, host, username, remote_dir="public_html/", 
                   local_dir="./", ssh_key_path=None):
    """Upload files to SSH server using scp"""
    
    try:
        for filename in file_array:
            local_path = os.path.join(local_dir, filename)
            
            # Check if local file exists
            if not os.path.exists(local_path):
                print(f"✗ Local file {local_path} not found, skipping...")
                continue
            
            # Build remote path
            remote_path = f"{username}@{host}:{remote_dir}{filename}"
            
            # Build scp command
            cmd = ["scp"]
            if ssh_key_path:
                cmd.extend(["-i", ssh_key_path])
            cmd.extend([local_path, remote_path])
            
            print(f"Uploading {filename}...")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✓ {filename} uploaded")
            else:
                print(f"✗ Error uploading {filename}: {result.stderr}")
        
        print("Upload process completed!")
        
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
# download_abv_ssh(["music-complete.html"], "abvchorus.org", "abvch0")
# upload_abv_ssh(["dog.txt"], "abvchorus.org", "abvch0")

In [ ]:
remote_path = "/mnt/c/Users/mcken/OneDrive/chorus/"

In [ ]:
# Option 2: Using subprocess with rsync (More efficient for multiple files)

def download_abv_rsync(file_array, host, username, remote_dir="public_html/", 
                       local_dir="./", ssh_key_path=None):
    """Download files using rsync over SSH (more efficient)"""
    
    try:
        # Create local directory if it doesn't exist
        os.makedirs(local_dir, exist_ok=True)
        
        for filename in file_array:
            # Build remote path
            remote_path = f"{username}@{host}:{remote_dir}{filename}"
            
            # Build rsync command
            cmd = ["rsync", "-avz", "--progress"]
            if ssh_key_path:
                cmd.extend(["-e", f"ssh -i {ssh_key_path}"])
            cmd.extend([remote_path, local_dir])
            
            print(f"Downloading {filename} with rsync...")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✓ {filename} downloaded")
            else:
                print(f"✗ Error downloading {filename}: {result.stderr}")
        
        print("Rsync download completed!")
        
    except Exception as e:
        print(f"Error: {e}")

def upload_abv_rsync(file_array, host, username, remote_dir="public_html/", 
                     local_dir="./", ssh_key_path=None):
    """Upload files using rsync over SSH"""
    
    try:
        for filename in file_array:
            local_path = os.path.join(local_dir, filename)
            
            # Check if local file exists
            if not os.path.exists(local_path):
                print(f"✗ Local file {local_path} not found, skipping...")
                continue
            
            # Build remote path
            remote_path = f"{username}@{host}:{remote_dir}"
            
            # Build rsync command
            cmd = ["rsync", "-avz", "--progress"]
            if ssh_key_path:
                cmd.extend(["-e", f"ssh -i {ssh_key_path}"])
            cmd.extend([local_path, remote_path])
            
            print(f"Uploading {filename} with rsync...")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✓ {filename} uploaded")
            else:
                print(f"✗ Error uploading {filename}: {result.stderr}")
        
        print("Rsync upload completed!")
        
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
# download_abv_rsync(["music-complete.html"], "abvchorus.org", "abvch0")
# upload_abv_rsync(["dog.txt"], "abvchorus.org", "abvch0")

In [ ]:
# Option 3: Using paramiko library (Pure Python SSH)
# First install: pip install paramiko

def download_abv_paramiko(file_array, host, username, remote_dir="public_html/", 
                          local_dir="./", ssh_key_path="~/.ssh/id_rsa"):
    """Download files using paramiko SFTP (Pure Python)"""
    
    try:
        import paramiko
        
        # Create SSH client
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        # Connect using key authentication
        key_path = os.path.expanduser(ssh_key_path)
        ssh.connect(host, username=username, key_filename=key_path)
        
        # Create SFTP client
        sftp = ssh.open_sftp()
        
        # Create local directory if it doesn't exist
        os.makedirs(local_dir, exist_ok=True)
        
        for filename in file_array:
            remote_path = f"{remote_dir}{filename}"
            local_path = os.path.join(local_dir, filename)
            
            print(f"Downloading {filename}...")
            try:
                sftp.get(remote_path, local_path)
                print(f"✓ {filename} downloaded")
            except Exception as e:
                print(f"✗ Error downloading {filename}: {e}")
        
        # Close connections
        sftp.close()
        ssh.close()
        print("SFTP download completed!")
        
    except ImportError:
        print("✗ paramiko not installed. Run: pip install paramiko")
    except Exception as e:
        print(f"Error: {e}")

def upload_abv_paramiko(file_array, host, username, remote_dir="public_html/", 
                        local_dir="./", ssh_key_path="~/.ssh/id_rsa"):
    """Upload files using paramiko SFTP"""
    
    try:
        import paramiko
        
        # Create SSH client
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        # Connect using key authentication
        key_path = os.path.expanduser(ssh_key_path)
        ssh.connect(host, username=username, key_filename=key_path)
        
        # Create SFTP client
        sftp = ssh.open_sftp()
        
        for filename in file_array:
            local_path = os.path.join(local_dir, filename)
            
            # Check if local file exists
            if not os.path.exists(local_path):
                print(f"✗ Local file {local_path} not found, skipping...")
                continue
            
            remote_path = f"{remote_dir}{filename}"
            
            print(f"Uploading {filename}...")
            try:
                sftp.put(local_path, remote_path)
                print(f"✓ {filename} uploaded")
            except Exception as e:
                print(f"✗ Error uploading {filename}: {e}")
        
        # Close connections
        sftp.close()
        ssh.close()
        print("SFTP upload completed!")
        
    except ImportError:
        print("✗ paramiko not installed. Run: pip install paramiko")
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
# download_abv_paramiko(["music-complete.html"], "abvchorus.org", "abvch0")
# upload_abv_paramiko(["dog.txt"], "abvchorus.org", "abvch0")

## Comparison of SSH Methods

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| **SCP** | Simple, reliable, built-in | Individual file transfers | Single files, simple setups |
| **Rsync** | Efficient, resume capability, compression | Requires rsync on both ends | Multiple files, large files |
| **Paramiko** | Pure Python, programmatic control | Extra dependency | Complex SSH operations |

## Usage Examples

```python
# Using default SSH key (~/.ssh/id_rsa)
download_abv_ssh(["music-complete.html"], "server.com", "username")

# Using specific SSH key
upload_abv_ssh(["file.txt"], "server.com", "username", ssh_key_path="/path/to/key")

# Multiple files with rsync
download_abv_rsync(["file1.txt", "file2.html", "file3.css"], 
                   "server.com", "username", remote_dir="public_html/")
```

## SSH Key Setup (if needed)

```bash
# Generate SSH key pair (if you don't have one)
ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

# Copy public key to remote server
ssh-copy-id username@server.com

# Test connection
ssh username@server.com
```

## Windows SSH Path Formats

The path format depends on which SSH server is running on the Windows machine:

### 1. **OpenSSH for Windows** (Built into Windows 10/11)
Uses **forward slashes** and Windows-style drive letters:
```python
remote_dir = "C:/Users/mcken/Documents/"
remote_dir = "D:/webfiles/public_html/"
```

### 2. **WSL (Windows Subsystem for Linux)**
Uses **Linux-style paths** with `/mnt/` prefix:
```python
remote_dir = "/mnt/c/Users/mcken/Documents/"
remote_dir = "/mnt/d/webfiles/public_html/"
```

### 3. **Cygwin SSH**
Uses **POSIX-style paths**:
```python
remote_dir = "/cygdrive/c/Users/mcken/Documents/"
remote_dir = "/cygdrive/d/webfiles/public_html/"
```

### 4. **PowerShell SSH (rare)**
Uses **Windows-style backslashes** (but escaped):
```python
remote_dir = "C:\\Users\\mcken\\Documents\\"
```

## How to Identify the SSH Server on Windows

### 1. **Check Service Name (Remote Desktop or Local Access)**
- Open **Services** (`services.msc`) and look for:
  - `OpenSSH SSH Server` (Windows built-in)
  - `Cygwin SSHD`
  - `sshd` (WSL or other)

### 2. **Check Process Name (Task Manager or Command Line)**
- Look for running processes:
  - `sshd.exe` (OpenSSH)
  - `cygwin-sshd.exe` (Cygwin)
  - `wsl.exe` (WSL)

### 3. **Check Version After SSH Login**
After connecting via SSH, run:
```bash
ssh username@windows-host
# Then run:
sshd -V
# Or:
ps -W | findstr sshd
```

### 4. **Check Welcome Banner**
When you connect via SSH, the banner may show:
- `OpenSSH_for_Windows` (OpenSSH)
- `CYGWIN_NT` (Cygwin)
- `WSL` (WSL)

### 5. **Check Default Paths**
- Try listing `/mnt/c/` (WSL), `/cygdrive/c/` (Cygwin), or `C:/` (OpenSSH)

### 6. **Ask the Administrator**
If you have access, ask which SSH server was installed.

---

**Tip:** The most common on modern Windows is OpenSSH (built-in since Windows 10/11).

In [ ]:
# curl ifconfig.me from vivo
home_machine = '141.154.9.231'

scp -r /home/tryit/public_html/abv/test.txt 141.154.9.231:"C:\Users\mcken\OneDrive\chorus\abv\tech\alpha"